# import

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Data

In [2]:
amazon= pd.read_csv('/Users/t-2024-m0196/my-repo/Git/Git/실전프로젝트/amazon_이상치처리.csv')

In [4]:
# 날짜 필터링 

start_date= '2017-01-01'
end_date= '2018-03-21'

amazon['DateKey'] = pd.to_datetime(amazon['DateKey'], format= '%m/%d/%Y')
amazon = amazon[(amazon['DateKey'] >=start_date) & (amazon['DateKey'] <= end_date)]
# sorted(amazon['DateKey'].unique())

amazon.info()
amazon.head(3)


<class 'pandas.core.frame.DataFrame'>
Index: 36307 entries, 27916 to 64222
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Custkey                           36307 non-null  int64         
 1   DateKey                           36307 non-null  datetime64[ns]
 2   Discount Amount                   36307 non-null  float64       
 3   Invoice Date                      36307 non-null  object        
 4   Invoice Number                    36307 non-null  int64         
 5   Item                              36307 non-null  object        
 6   Line Number                       36307 non-null  int64         
 7   List Price                        36307 non-null  float64       
 8   Order Number                      36307 non-null  int64         
 9   Sales Amount                      36307 non-null  float64       
 10  Sales Amount Based on List Price  36307 non-nul

,Custkey,DateKey,Discount Amount,Invoice Date,Invoice Number,Item,Line Number,List Price,Order Number,Sales Amount,Sales Amount Based on List Price,Sales Cost Amount,Sales Margin Amount,Sales Price,Sales Quantity,Sales Rep,U/M
27916,10008602,2018-03-21,439.2400,2018/03/21,226437,Tell Tale Summer Squash,21000,195.0000,320075,535.76,975.0000,372.26,163.50,107.152000,5,103,EA
27917,10008602,2018-03-21,1517.3076,2018/03/21,226437,Nationeel Avocado Dip,13000,842.0244,320075,1850.79,3368.0976,1256.20,594.59,462.697500,4,103,EA
27918,10014779,2018-03-21,192.4900,2018/03/21,226451,Super White Sugar,9000,157.7600,321178,280.79,473.2800,143.73,137.06,93.596667,3,105,SE


In [5]:
temp= amazon['Custkey'].value_counts()
print(len(temp[temp > 1]))
# 34292 개의 Custkey 데이터 중에 중복구매한 사용자 수 482
print(len(temp[temp==1]))
# 1인 사람 41개
print(len(temp))

487
42
529


In [6]:
print(amazon.shape)
amazon.isna().sum()

(36307, 17)


Custkey                             0
DateKey                             0
Discount Amount                     0
Invoice Date                        0
Invoice Number                      0
Item                                0
Line Number                         0
List Price                          0
Order Number                        0
Sales Amount                        0
Sales Amount Based on List Price    0
Sales Cost Amount                   0
Sales Margin Amount                 0
Sales Price                         0
Sales Quantity                      0
Sales Rep                           0
U/M                                 0
dtype: int64

In [7]:
print('총 매출: ', round(amazon['Sales Amount'].sum(), 2))
print('총 할인 금액: ', round(amazon['Discount Amount'].sum(), 2))
print('총 주문 수: ', round(amazon['Order Number'].sum(), 2))

총 매출:  96803974.11
총 할인 금액:  69436945.25
총 주문 수:  8375355093


# 고객별 Sales Amount

In [8]:
cust_m= amazon.groupby('Custkey')['Sales Amount'].sum().reset_index()

cust_m.sort_values(by= 'Sales Amount', ascending= False)

,Custkey,Sales Amount
343,10021485,10949474.37
432,10025024,7951267.71
443,10025052,5419724.38
440,10025039,3259962.46
489,10025919,3250786.04
...,...,...
491,10026009,224.55
272,10019812,221.04
427,10024924,216.33
169,10013039,216.20


In [9]:
# R, F

cust_rf = amazon.groupby('Custkey').agg(
    Last_Purchase=('Invoice Date', 'max')
    , Frequency=('Order Number', 'nunique') # invoice number?
).reset_index()

cust_rf

,Custkey,Last_Purchase,Frequency
0,10000453,2017/01/06,2
1,10000455,2017/11/04,12
2,10000456,2017/12/04,12
3,10000457,2017/12/01,3
4,10000458,2018/03/06,3
...,...,...,...
524,10027381,2018/03/14,21
525,10027560,2017/12/26,15
526,10027572,2018/03/20,12
527,10027575,2018/03/06,4


# RFM 523개

In [10]:
cust_rfm= cust_rf.merge(cust_m, on= 'Custkey')

cust_rfm= cust_rfm.rename(columns= {'Last_Purchase': 'Recency',
                          'Sales Amount': 'Monetary'})

In [11]:
cust_rfm.info()
cust_rfm.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Custkey    529 non-null    int64  
 1   Recency    529 non-null    object 
 2   Frequency  529 non-null    int64  
 3   Monetary   529 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 16.7+ KB


,Custkey,Recency,Frequency,Monetary
0,10000453,2017/01/06,2,8422.15
1,10000455,2017/11/04,12,22403.73
2,10000456,2017/12/04,12,104994.68


In [12]:
# sorted(cust_rfm['Recency'].unique())
sorted(cust_rfm['Monetary'].unique())

# 소수점 X
cust_rfm['Monetary']= round(cust_rfm['Monetary'])

# 데이터타입 int로 변경
cust_rfm['Monetary']= cust_rfm['Monetary'].astype(int)

In [13]:
# recency datetime으로 변환
cust_rfm['Recency']= pd.to_datetime(cust_rfm['Recency'])

# 2018 03 22을 현재날짜로 설정, 가장 최근의 주문 일자를 뺌
cust_rfm['Recency']= dt.datetime(2018, 3, 22) - cust_rfm['Recency']
# 01로 하니까 오류남

# +1
cust_rfm['Recency']= cust_rfm['Recency'].apply(lambda x:x.days+1)

cust_rfm

,Custkey,Recency,Frequency,Monetary
0,10000453,441,2,8422
1,10000455,139,12,22404
2,10000456,109,12,104995
3,10000457,112,3,14285
4,10000458,17,3,4994
...,...,...,...,...
524,10027381,9,21,70995
525,10027560,87,15,48483
526,10027572,3,12,46291
527,10027575,17,4,3549


# CSV 파일 생성

In [14]:
cust_rfm.to_csv('cust_rfm.csv', index= False)